# Arxiv Explorer Tools - minimal TF-IDF Vector Search
- extract articles on topics of interest from the too-many-to-look-through loads of articles that come out each day.
- saves results to json and html
- minimal TF-IDF is vanilla python (no additional packages or libraries)
- arxiv reading uses 'beautiful soup'
- various classic distance metrics use:
    - scikit-learn
    - scipy
    - numpy

### Setup & Install:
- have python installed and use an python env
- use a jupyter notebook or script, etc.


## Note:
- This is more portable than an embedding model but it is just as slow.
- In theory distance search can be used in the vector space, in reality it is not good enough to use.

  


- https://pypi.org/project/beautifulsoup4/

requirements.txt ->
```
scikit-learn
scipy
numpy
beautifulsoup4
```

In [1]:
from datetime import datetime

start_time_whole_single_task = datetime.now()
end_time_whole_single_task = datetime.now()


def duration_min_sec(start_time, end_time):

    duration = end_time - start_time

    duration_seconds = duration.total_seconds()

    minutes = int(duration_seconds // 60)
    seconds = duration_seconds % 60
    time_message = f"{minutes}_min__{seconds:.1f}_sec"

    return time_message

duration_time = duration_min_sec(start_time_whole_single_task, end_time_whole_single_task)
print(f"Duration to run -> {duration_time}")

Duration to run -> 0_min__0.0_sec


In [2]:
# step 1: make corpus vector-matrix
# step 2: get vector of the search-phrase
# step 3: get vector of  each text
# step 4: get scores
# step 5: evaluates if score is succss or fail
# step 6: if success: do stuff with text, else: move on

# minimal weighted matching

In [3]:
# # import math
# # from collections import Counter


# # And an even more simplistic basic key word search (with optional weights)

# import re

# def rank_documents_on_weighted_matches(documents, keyword_weights):
#     """
#     Ranks documents based on the presence of weighted keywords-phrases.
#     comparison looks at text without:
#     - captialization
#     - spaces
#     - newlines
#     - special symbols

#     Parameters:
#     documents (list of str): The list of documents to be ranked.
#     keyword_weights (list of tuple): A list of tuples, where the first element is the keyword and the
#     second element is the corresponding weight.

#     Returns:
#     list of (str, float): A list of tuples, where the first element is the document and the
#     second element is the ranking score.
#     """

#     ranked_documents = []

#     for document in documents:
#         score = 0
#         # Make the document lowercase and strip all symbols, spaces, and newline characters
#         match_document = re.sub(r'[^\w\s]', '', document.lower()).replace('\n', '').replace(' ','')
#         # print(match_document)
#         for keyword, weight in keyword_weights:

#             # Make the keyword lowercase and strip all symbols, spaces, and newline characters
#             match_keyword = re.sub(r'[^\w\s]', '', keyword.lower()).replace('\n', '').replace(' ','')
#             # print(match_keyword)
#             # Check if the keyword-phrase is in the document
#             if match_keyword in match_document:
#                 # If the keyword-phrase is in the document, add its weight to the score
#                 score += weight

#         ranked_documents.append((document, score))

#     # Sort the documents by their ranking scores in descending order
#     ranked_documents.sort(key=lambda x: x[1], reverse=True)

#     return ranked_documents


# ################
# # Example usage
# ################
# documents = [
#     "This is the first document about machine learning.",
#     "The second document discusses data analysis and visualization.",
#     "The third document focuses on natural language processing.",
#     "The fourth document talks about deep learning and neural networks.",
#     """to test line breaks
#     Emotion mining
#      data
#     analysis
#     Keywords: emotion mining, sentiment analysis, natural disasters, psychology, technological disasters""",
# ]

# keyword_weights = [("machine learning", 3), ("data analysis", 2), ("natural language processing", 4), ("deep learning", 5), ("neural networks", 6)]

# ranked_documents = rank_documents_on_weighted_matches(documents, keyword_weights)

# for document, score in ranked_documents:
#     print(f"Document: {document}\nScore: {score}\n")


# Arxiv Explorerer


In [4]:
###################
# Arxiv Explorerer
###################

# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text


import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime


start_time_whole_single_task = datetime.now()


# ##########################################
# # Make comparison phrase and vectorize it
# ##########################################
# comparison_phrase = "computer vision resolution enhancement"
# # comparison_phrase = "cyber security"
# # comparison_phrase = "natural language processing"


# Get Article Corpus

In [5]:
start_segment_time = datetime.now()

#####################
# Get Article Corpus
#####################

# List to hold all article data
article_data = []

# # Make a request to the website
r = requests.get('https://arxiv.org/list/cs/new')

url = "https://arxiv.org/list/cs/new"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# # Find all the articles
articles = soup.find_all('dt')

# # Find all the titles
articles_title = soup.find_all('div', {'class': 'list-title mathjax'})

# Find all the subject on the page
articles_subject = soup.find_all('dd')


###############
# make corpus
###############

corpus = []
report_list = []
article_dicts = []

for this_index, article in enumerate(articles):

    ################################################
    # Extract each field of data about each article
    ################################################

    # Extract the title
    title = articles_title[this_index].text.split('Title:')[1].strip()

    # Extract the subjects
    subjects = articles_subject[this_index].find('span', {'class': 'primary-subject'}).text

    arxiv_id = article.find('a', {'title': 'Abstract'}).text.strip()

    abstract_p = article.find_next_sibling('dd').find('p', {'class': 'mathjax'})

    # Extract the abstract
    if abstract_p:
        abstract = abstract_p.text.strip()
    else:
        abstract = ""

    pdf_link_segment = article.find('a', {'title': 'Download PDF'})['href']

    arxiv_id = article.find('a', {'title': 'Abstract'}).text.strip()
    pdf_link = f"https://arxiv.org{pdf_link_segment}"
    paper_link = f"https://arxiv.org/abs/{arxiv_id[6:]}"

    extracted_article_string = title + " " + abstract + " " + str(subjects)

    # assemble corpus
    article_characters = ""

    article_characters += f"'arxiv_id': {arxiv_id}, "
    article_characters += f"'paper_link': {paper_link}, "
    article_characters += f"'pdf_link': {pdf_link}, "

    article_characters += title + " "
    article_characters += subjects + " "
    article_characters += abstract

    ##################################
    # Make Bundles (sharing an index)
    ##################################

    # add to corpus: just the meaningful text
    corpus.append(extracted_article_string)

    # add to simple report_list: includes link and article ID info
    report_list.append(article_characters)

    # Append the data to the list
    article_dicts.append({
        'title': title,
        'abstract': abstract,
        'paper_link': paper_link,
        'pdf_link': pdf_link,
        'subjects': subjects,
        'arxiv_id': arxiv_id,
    })

    # using this because only basic search works
    corpus = report_list


# # Segment Timer
# start_segment_time = datetime.now()
end_segment_time = datetime.now()
duration_time = duration_min_sec(start_segment_time, end_segment_time)
print(f"Duration to run segment -> {duration_time}")

Duration to run segment -> 0_min__3.5_sec


In [6]:
# inspection (size of corpus)
len(corpus)

1221

In [7]:
# inspection (size of report_list)
len(report_list)

1221

In [8]:
# inspection (sample of corpus)
corpus[0]

"'arxiv_id': arXiv:2407.17471, 'paper_link': https://arxiv.org/abs/2407.17471, 'pdf_link': https://arxiv.org/pdf/2407.17471, Real-Time Automated donning and doffing detection of PPE based on Yolov4-tiny Computer Vision and Pattern Recognition (cs.CV) Maintaining patient safety and the safety of healthcare workers (HCWs) in hospitals and clinics highly depends on following the proper protocol for donning and taking off personal protective equipment (PPE). HCWs can benefit from a feedback system during the putting on and removal process because the process is cognitively demanding and errors are common. Centers for Disease Control and Prevention (CDC) provided guidelines for correct PPE use which should be followed. A real time object detection along with a unique sequencing algorithms are used to identify and determine the donning and doffing process in real time. The purpose of this technical research is two-fold: The user gets real time alert to the step they missed in the sequence if

In [9]:
# inspection (sample of report_list)
report_list[0]

"'arxiv_id': arXiv:2407.17471, 'paper_link': https://arxiv.org/abs/2407.17471, 'pdf_link': https://arxiv.org/pdf/2407.17471, Real-Time Automated donning and doffing detection of PPE based on Yolov4-tiny Computer Vision and Pattern Recognition (cs.CV) Maintaining patient safety and the safety of healthcare workers (HCWs) in hospitals and clinics highly depends on following the proper protocol for donning and taking off personal protective equipment (PPE). HCWs can benefit from a feedback system during the putting on and removal process because the process is cognitively demanding and errors are common. Centers for Disease Control and Prevention (CDC) provided guidelines for correct PPE use which should be followed. A real time object detection along with a unique sequencing algorithms are used to identify and determine the donning and doffing process in real time. The purpose of this technical research is two-fold: The user gets real time alert to the step they missed in the sequence if

# Vector Model: TF-IDF
- olde schoole

In [10]:
query = "computer vision"

In [11]:
start_segment_time = datetime.now()
"""
vanilla_TF-IDF_v8.ipynb

This notebook is based on:
- https://medium.com/@coldstart_coder/understanding-and-implementing-tf-idf-in-python-a325d1301484
- https://www.kaggle.com/code/tylerpoff/understanding-and-implementing-tf-idf-in-python/notebook


instructions:
1. set query string
2. set corpus list of strings
3. create TF-IDF vector-matrix (pick an inverse_document_frequency variant)
4. search/sort for top-N results: tfidf_vector_search_top_n()
5. print results etc.

"""

import math
import time


"""# query"""
query = query

corpus_unsplit = corpus



def term_frequency(word, document):
    return document.count(word) / len(document)
tf = term_frequency

# # non-plus-1 variant ("unsafe" variant)
# def inverse_document_frequency_unsafe(word, corpus):
#     count_of_documents = len(corpus)
#     count_of_documents_with_word = sum([1 for doc in corpus if word in doc])
#     idf = math.log10(count_of_documents/count_of_documents_with_word)
#     return idf

# sklearn variant
def inverse_document_frequency(word, corpus):
    count_of_documents = len(corpus) + 1
    # count_of_documents_with_word = sum([1 for doc in corpus if word in doc]) + 1
    count_of_documents_with_word = 0

    for doc in corpus:
        count_of_documents_with_word
        # print(f"doc -> {doc}")
        # print(f"word -> {word}")

        if word in doc:
            # print(f"count_of_documents_with_word -> {count_of_documents_with_word}")
            count_of_documents_with_word += 1

    idf = math.log10(count_of_documents/count_of_documents_with_word) + 1
    return idf

idf = inverse_document_frequency

def TF_IDF(word, document, corpus):
    return tf(word, document) * idf(word, corpus)


"""# corpus of documents"""
split_corpus = [c.split() for c in corpus_unsplit]
num_documents = len(split_corpus)


"""### Optional Sample Target Word Analysis"""

"""
optional
"""
# target_word = "Expert"

# print("searching for the word '%s'"%target_word)
# for i, document in enumerate(split_corpus):
#     tf_score = tf(target_word, document)
#     idf_score = idf(target_word, split_corpus)
#     tf_idf_score = TF_IDF(target_word, document, split_corpus)

#     print("document %s: '%s'\n    tf score: %s\n    idf score: %s\n    tf_idf score:%s"%(i, document, tf_score, idf_score, tf_idf_score))
#     print("-"*30)




"""## word to vector mappings"""

"""
 create the word to vector mappings,
 we want each word to map to a unique point in our word vectors.
combine the complete corpus into a single list of words; remove duplicates.
use position in this list as the index for a word vector
"""
word_set = list(set(sum(split_corpus, [])))
# create a lookup for each word to it's index,
word_to_index = {word:i for i, word in enumerate(word_set)}

num_words = len(word_set)


import math

def get_tfidf_vector(query, word_set, split_corpus, word_to_index):

    # Create an empty list to store the word vectors
    word_vectors = []

    # Calculate the TF-IDF score for each word in the query
    query_keywords = query.split()
    query_vector = [0 for _ in range(len(word_set))]
    for word in query_keywords:
        if word in word_set:
            word_index = word_to_index[word]
            query_vector[word_index] = TF_IDF(word, query_keywords, split_corpus)

    return query_vector



"""## create the word vectors"""
# create an empty list to store our word vectors
word_vectors = []
for document in split_corpus:
    # for our new document create a new word vector
    new_word_vector = [0 for i in range(num_words)]

    # now we loop through each word in our document and compute the tf-idf score and populate our vector with it,
    # we only care about words in this document because words outside of it will remain zero
    for word in document:
        # get the score
        tf_idf_score = TF_IDF(word, document, split_corpus)
        # next get the index for this word in our word vector
        word_index = word_to_index[word]
        # populate the vector
        new_word_vector[word_index] = tf_idf_score

    # add new word vector to list of existing word_vectors
    word_vectors.append(new_word_vector)


"""## one word vector in comparision to document"""
# # inspection
# print(corpus_unsplit[0])
# print(word_vectors[0])


#############
# Exmple Use
#############

########################################
## Searching with TF-IDF Sparse Vectors
########################################
query_keywords = query.split()

# now we loop through each documents word vector, get the tf-idf score for each keyword, sum them up and that is our tf-idf for that document,
# we keep track of the best document and return that as our result,
tf_idf_scores = []
best_document_index = 0
best_tf_idf = 0

for i, word_vector in enumerate(word_vectors):
    document_tf_idf_score_for_query = 0
    for word in query_keywords:
        # first do a check, does this word appear in our split_corpus of documents?
        # if not skip this keyword
        if word not in word_set:
            continue

        # get the index for this keyword and directly pull it from the word vector
        word_index = word_to_index[word]
        document_tf_idf_score_for_query += word_vector[word_index]
    tf_idf_scores.append(document_tf_idf_score_for_query) # keep track of all tf_idf scores, just in case we want to review them,


    # # does this tf_idf score for this document beat our previous best?
    # if document_tf_idf_score_for_query > best_tf_idf:
    #     best_tf_idf = document_tf_idf_score_for_query
    #     best_document_index = i



# Inspection & Study

# from pprint import pprint
# # then print out our results
# # print("results of query: ", query)
# print("best tf_idf score sum for query: ", best_tf_idf)
# print("best document: ", corpus_unsplit[best_document_index])
# print("complete list of tf_idf scores: ", tf_idf_scores)
# from pprint import pprint
# print("tf_idf_scores -> ")
# pprint(tf_idf_scores)
##  pprint(corpus_unsplit)



def tfidf_vector_search_top_n(query, corpus, n):
    query_keywords = query.split()

    tf_idf_scores = []
    for i, word_vector in enumerate(word_vectors):
        document_tf_idf_score_for_query = 0
        for word in query_keywords:
            if word not in word_set:
                continue

            word_index = word_to_index[word]
            document_tf_idf_score_for_query += word_vector[word_index]
        tf_idf_scores.append((document_tf_idf_score_for_query, i))

    # Sort the TF-IDF scores in descending order
    tf_idf_scores.sort(reverse=True)

    # Extract the document indices from the top-N results
    top_n_document_indices = [index for _, index in tf_idf_scores[:n]]

    # Return the top-N documents and their TF-IDF scores
    top_n_documents = [corpus[index] for index in top_n_document_indices]
    top_n_tf_idf_scores = [score for score, _ in tf_idf_scores[:n]]

    return top_n_documents, top_n_tf_idf_scores


# # Set This
# how_many_results = 5

# # Search
# top_n = how_many_results
# start_time = time.monotonic()  # timer
# top_n_documents, top_n_tf_idf_scores = tfidf_vector_search_top_n(query, corpus_unsplit, top_n)
# end_time = time.monotonic()  # timer
# elapsed_time = end_time - start_time  # timer

# print(f"Top-{top_n} results for query: {query}")
# for i, (document, score) in enumerate(zip(top_n_documents, top_n_tf_idf_scores)):
#     print(f"Result {i+1}:")
#     print(f"TF-IDF score: {score}")
#     print(f"Document: {document}\n")
# # timer
# print(f"Elapsed time: {elapsed_time} seconds")



# # Segment Timer
# start_segment_time = datetime.now()
end_segment_time = datetime.now()
duration_time = duration_min_sec(start_segment_time, end_segment_time)
print(f"Duration to run segment -> {duration_time}")

Duration to run segment -> 12_min__14.1_sec


In [12]:
# # # inspection
# print(corpus_unsplit[0])
# print(word_vectors[0])

In [13]:
def get_tfidf_vector(query, word_set, split_corpus, word_to_index):

    # Create an empty list to store the word vectors
    word_vectors = []

    # Calculate the TF-IDF score for each word in the query
    query_keywords = query.split()
    query_vector = [0 for _ in range(len(word_set))]
    for word in query_keywords:
        if word in word_set:
            word_index = word_to_index[word]
            query_vector[word_index] = TF_IDF(word, query_keywords, split_corpus)

    return query_vector


# Simple Top-N TF-IDF

In [15]:
# Set This
how_many_results = 10

# Search
top_n = how_many_results
start_tfidf_time = time.monotonic()  # timer
top_n_documents, top_n_tf_idf_scores = tfidf_vector_search_top_n(query, corpus_unsplit, top_n)
end_tfidf_time = time.monotonic()  # timer
elapsed_time = end_tfidf_time - start_tfidf_time  # timer

print(f"Top-{top_n} results for query: {query}")
for i, (document, score) in enumerate(zip(top_n_documents, top_n_tf_idf_scores)):
    print(f"Result {i+1}:")
    print(f"TF-IDF score: {score}")
    print(f"Document: {document}\n")
# timer
print(f"Elapsed time: {elapsed_time} seconds")

Top-10 results for query: computer vision
Result 1:
TF-IDF score: 0.08576801905166966
Document: Mpox Detection Advanced: Rapid Epidemic Response Through Synthetic Data Rapid development of disease detection models using computer vision is crucial in responding to medical emergencies, such as epidemics or bioterrorism events. Traditional data collection methods are often too slow in these scenarios, requiring innovative approaches for quick, reliable model generation from minimal data. Our study introduces a novel approach by constructing a comprehensive computer vision model to detect Mpox lesions using only synthetic data. Initially, these models generated a diverse set of synthetic images representing Mpox lesions on various body parts (face, back, chest, leg, neck, arm) across different skin tones as defined by the Fitzpatrick scale (fair, brown, dark skin). Subsequently, we trained and tested a vision model with this synthetic dataset to evaluate the diffusion models' efficacy in p

In [16]:
# Set This
how_many_results = 10

# Search
top_n = how_many_results
start_tfidf_time = time.monotonic()  # timer
top_n_documents, top_n_tf_idf_scores = tfidf_vector_search_top_n(query, report_list, top_n)
end_tfidf_time = time.monotonic()  # timer
elapsed_time = end_tfidf_time - start_tfidf_time  # timer

print(f"Top-{top_n} results for query: {query}")
for i, (document, score) in enumerate(zip(top_n_documents, top_n_tf_idf_scores)):
    print(f"Result {i+1}:")
    print(f"TF-IDF score: {score}")
    print(f"Document: {document}\n")
# timer
print(f"Elapsed time: {elapsed_time} seconds")

Top-10 results for query: computer vision
Result 1:
TF-IDF score: 0.08576801905166966
Document: Mpox Detection Advanced: Rapid Epidemic Response Through Synthetic Data Rapid development of disease detection models using computer vision is crucial in responding to medical emergencies, such as epidemics or bioterrorism events. Traditional data collection methods are often too slow in these scenarios, requiring innovative approaches for quick, reliable model generation from minimal data. Our study introduces a novel approach by constructing a comprehensive computer vision model to detect Mpox lesions using only synthetic data. Initially, these models generated a diverse set of synthetic images representing Mpox lesions on various body parts (face, back, chest, leg, neck, arm) across different skin tones as defined by the Fitzpatrick scale (fair, brown, dark skin). Subsequently, we trained and tested a vision model with this synthetic dataset to evaluate the diffusion models' efficacy in p

# Get Article Corpus

In [17]:

# #############
# # Write Data
# #############

# # Posix UTC Seconds
# # make readable time
# from datetime import datetime, UTC
# date_time = datetime.now(UTC)
# clean_timestamp = date_time.strftime('%Y-%m-%d__%H%M%S%f')


# # Save the data to a JSON file
# with open(f'articles_{clean_timestamp}.json', 'w') as f:
#     json.dump(article_data, f)


# # Create an HTML file
# html = '<html><body>'
# for article in article_data:
#     html += f'<h2><a href="{article["paper_link"]}">{article["title"]}</a></h2>'
#     html += f'<p>{article["abstract"]}</p>'
#     html += f'<p>Subjects: ", {str(article["subjects"])}</p>'

#     html += f'<a href="{article["paper_link"]}">{article["paper_link"]}</a>'
#     html += f'<p>paper link: ", {str(article["paper_link"])}</p>'

#     html += f'<a href="{article["pdf_link"]}">{article["pdf_link"]}</a>'
#     html += f'<p>pdf link: ", {str(article["pdf_link"])}</p>'

#     html += f'<p>arxiv id: ", {str(article["arxiv_id"])}</p>'

# html += '</body></html>'


# # Save the HTML to a file
# with open(f'articles{clean_timestamp}.html', 'w') as f:
#     f.write(html)

# # Duration time print
# end_time_whole_single_task = datetime.now()
# duration_time = duration_min_sec(start_time_whole_single_task, end_time_whole_single_task)
# print(f"Duration to run -> {duration_time}")


In [18]:
duration_time

'12_min__14.1_sec'